<a href="https://colab.research.google.com/github/Masciel-Sevilla/Segmentacion/blob/main/Filtro_Small.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [8]:
# ==============================================================================
# PASO 1: INSTALAR LIBRERÍAS (si no está instalada)
# ==============================================================================
!pip install git+https://github.com/lucasb-eyer/pydensecrf.git

# ==============================================================================
# PASO 2: IMPORTAR LIBRERÍAS
# ==============================================================================
import tensorflow as tf
from tensorflow.keras import layers
import numpy as np
import os
from glob import glob
import pydensecrf.densecrf as dcrf
from pydensecrf.utils import unary_from_softmax, create_pairwise_bilateral, create_pairwise_gaussian
import traceback

# ==============================================================================
# PASO 3: CONFIGURACIÓN Y RUTAS
# ==============================================================================
IMG_HEIGHT = 128
IMG_WIDTH = 128
NUM_CLASSES = 6
CLASS_NAMES = ['Background', 'Cow-tongue', 'Dandelion', 'Kikuyo', 'Other', 'Potato']
MODEL_SAVE_PATH = 'efficient_weed_model_S_best.keras'
BASE_PATH = './Balanced'
# --- Se mantienen las rutas del conjunto de TEST para la evaluación final ---
EVAL_IMAGES_PATH = os.path.join(BASE_PATH, 'test/images')
EVAL_MASKS_PATH = os.path.join(BASE_PATH, 'test/masks')

# ==============================================================================
# PASO 4: DEFINICIONES COMPLETAS PARA CARGAR EL MODELO S
# ==============================================================================

# --- Clases personalizadas del modelo S ---
class ASPPModule(layers.Layer):
    def __init__(self, filters=192, **kwargs):
        super(ASPPModule, self).__init__(**kwargs)
        self.filters = filters
        self.conv_1x1 = layers.Conv2D(filters, 1, padding='same', use_bias=False)
        self.bn_1x1 = layers.BatchNormalization()
        self.relu_1x1 = layers.ReLU()
        self.conv_3x3_6 = layers.Conv2D(filters, 3, padding='same', dilation_rate=6, use_bias=False)
        self.bn_3x3_6 = layers.BatchNormalization()
        self.relu_3x3_6 = layers.ReLU()
        self.conv_3x3_12 = layers.Conv2D(filters, 3, padding='same', dilation_rate=12, use_bias=False)
        self.bn_3x3_12 = layers.BatchNormalization()
        self.relu_3x3_12 = layers.ReLU()
        self.conv_3x3_18 = layers.Conv2D(filters, 3, padding='same', dilation_rate=18, use_bias=False)
        self.bn_3x3_18 = layers.BatchNormalization()
        self.relu_3x3_18 = layers.ReLU()
        self.global_avg_pool = layers.GlobalAveragePooling2D(keepdims=True)
        self.conv_1x1_gap = layers.Conv2D(filters, 1, padding='same', use_bias=False)
        self.bn_1x1_gap = layers.BatchNormalization()
        self.relu_1x1_gap = layers.ReLU()
        self.conv_final = layers.Conv2D(filters, 1, padding='same', use_bias=False)
        self.bn_final = layers.BatchNormalization()
        self.relu_final = layers.ReLU()
        self.dropout = layers.Dropout(0.2)

    def call(self, inputs, training=None):
        input_shape = tf.shape(inputs)
        conv_1x1 = self.relu_1x1(self.bn_1x1(self.conv_1x1(inputs), training=training))
        conv_3x3_6 = self.relu_3x3_6(self.bn_3x3_6(self.conv_3x3_6(inputs), training=training))
        conv_3x3_12 = self.relu_3x3_12(self.bn_3x3_12(self.conv_3x3_12(inputs), training=training))
        conv_3x3_18 = self.relu_3x3_18(self.bn_3x3_18(self.conv_3x3_18(inputs), training=training))
        gap = self.global_avg_pool(inputs)
        gap = self.relu_1x1_gap(self.bn_1x1_gap(self.conv_1x1_gap(gap), training=training))
        gap = tf.image.resize(gap, [input_shape[1], input_shape[2]], method='bilinear')
        concat = layers.Concatenate()([conv_1x1, conv_3x3_6, conv_3x3_12, conv_3x3_18, gap])
        output = self.relu_final(self.bn_final(self.conv_final(concat), training=training))
        output = self.dropout(output, training=training)
        return output

    def get_config(self):
        config = super(ASPPModule, self).get_config()
        config.update({"filters": self.filters})
        return config

class DeformableAttention(layers.Layer):
    def __init__(self, filters, **kwargs):
        super(DeformableAttention, self).__init__(**kwargs)
        self.filters = filters
        self.attention_conv = layers.Conv2D(self.filters, 1, padding='same', activation='sigmoid', name='attention_weights_conv', use_bias=False)
        self.bn_attention = layers.BatchNormalization()
        self.feature_conv = layers.SeparableConv2D(self.filters, 3, padding='same', name='feature_processing_conv', use_bias=False)
        self.bn_feature = layers.BatchNormalization()
        self.relu_feature = layers.ReLU()

    def call(self, inputs, training=None):
        attention_weights = self.bn_attention(self.attention_conv(inputs), training=training)
        features = self.relu_feature(self.bn_feature(self.feature_conv(inputs), training=training))
        attended_features = features * attention_weights
        return attended_features

    def get_config(self):
        config = super(DeformableAttention, self).get_config()
        config.update({"filters": self.filters})
        return config

# --- Funciones personalizadas (placeholders para la carga del modelo) ---
def dice_coefficient(y_true, y_pred): return 0.0
def dice_loss(y_true, y_pred): return 0.0
def combined_loss(y_true, y_pred): return 0.0
def iou_metric(y_true, y_pred): return 0.0

# --- Función IoU para la Evaluación ---
def iou_per_class(y_true, y_pred):
    y_true = tf.cast(y_true, tf.float32)
    if y_pred.shape[-1] != NUM_CLASSES:
        y_pred = tf.one_hot(tf.cast(y_pred, tf.int32), depth=NUM_CLASSES)
    else:
        y_pred = tf.cast(y_pred, tf.float32)
    intersection = tf.reduce_sum(y_true * y_pred, axis=[1, 2])
    union = tf.reduce_sum(y_true, axis=[1, 2]) + tf.reduce_sum(y_pred, axis=[1, 2]) - intersection
    iou = tf.where(tf.equal(union, 0), 1.0, intersection / union)
    return tf.squeeze(iou)

# --- Funciones de Post-Procesamiento (CRF y TTA) ---
def refine_segmentation_with_crf(image, softmax_output, crf_params):
    image = np.ascontiguousarray(image)
    unary = unary_from_softmax(np.ascontiguousarray(softmax_output))
    d = dcrf.DenseCRF2D(image.shape[1], image.shape[0], softmax_output.shape[0])
    d.setUnaryEnergy(unary)
    d.addPairwiseEnergy(create_pairwise_gaussian(sdims=(crf_params['g_sdims'], crf_params['g_sdims']), shape=image.shape[:2]),
                        compat=crf_params['g_compat'], kernel=dcrf.DIAG_KERNEL, normalization=dcrf.NORMALIZE_SYMMETRIC)
    d.addPairwiseEnergy(create_pairwise_bilateral(sdims=(crf_params['b_sdims'], crf_params['b_sdims']),
                                                  schan=(crf_params['b_rgb_sdims'], crf_params['b_rgb_sdims'], crf_params['b_rgb_sdims']),
                                                  img=image, chdim=2),
                        compat=crf_params['b_compat'], kernel=dcrf.DIAG_KERNEL, normalization=dcrf.NORMALIZE_SYMMETRIC)
    Q = d.inference(crf_params['iterations'])
    return np.argmax(Q, axis=0).reshape((image.shape[0], image.shape[1]))

def perform_segmentation_tta(model, image_tensor):
    all_predictions = []
    pred_original = model.predict(image_tensor, verbose=0)
    all_predictions.append(pred_original)
    flipped_lr = tf.image.flip_left_right(image_tensor)
    pred_flipped_lr = model.predict(flipped_lr, verbose=0)
    all_predictions.append(tf.image.flip_left_right(pred_flipped_lr))
    flipped_ud = tf.image.flip_up_down(image_tensor)
    pred_flipped_ud = model.predict(flipped_ud, verbose=0)
    all_predictions.append(tf.image.flip_up_down(pred_flipped_ud))
    avg_preds = tf.reduce_mean(tf.stack(all_predictions), axis=0)
    return avg_preds[0]

# ==============================================================================
# PASO 5: CARGAR DATOS Y EJECUTAR EVALUACIÓN
# ==============================================================================
def load_dataset_for_evaluation(img_path, msk_path):
    image_paths = sorted(glob(os.path.join(img_path, '*.jpg')))
    mask_paths = sorted(glob(os.path.join(msk_path, '*.png')))
    if not image_paths or not mask_paths:
        raise FileNotFoundError(f"No se encontraron imágenes o máscaras en: {img_path}, {msk_path}")
    val_ds_original = tf.data.Dataset.from_tensor_slices(image_paths).map(lambda p: tf.image.resize(tf.image.decode_jpeg(tf.io.read_file(p), channels=3), [IMG_HEIGHT, IMG_WIDTH]))
    val_ds_processed = val_ds_original.map(tf.keras.applications.efficientnet_v2.preprocess_input)
    val_ds_masks = tf.data.Dataset.from_tensor_slices(mask_paths).map(lambda p: tf.one_hot(tf.squeeze(tf.cast(tf.image.resize(tf.image.decode_png(tf.io.read_file(p), channels=1), [IMG_HEIGHT, IMG_WIDTH], method='nearest'), tf.int32)), depth=NUM_CLASSES))
    return tf.data.Dataset.zip((val_ds_original, val_ds_processed, val_ds_masks)).batch(1)

if __name__ == "__main__":
    custom_objects = {
        'combined_loss': combined_loss, 'dice_coefficient': dice_coefficient,
        'iou_metric': iou_metric, 'dice_loss': dice_loss,
        'ASPPModule': ASPPModule, 'DeformableAttention': DeformableAttention
    }

    try:
        model = tf.keras.models.load_model(MODEL_SAVE_PATH, custom_objects=custom_objects)
        print(f"✅ Modelo '{MODEL_SAVE_PATH}' cargado exitosamente.")

        print(f"📖 Cargando datos del conjunto de TEST desde: {EVAL_IMAGES_PATH}")
        eval_dataset = load_dataset_for_evaluation(EVAL_IMAGES_PATH, EVAL_MASKS_PATH)

        # --- Listas para almacenar los resultados de los TRES métodos ---
        per_class_iou_normal_list, per_class_iou_crf_list, per_class_iou_tta_list = [], [], []

        crf_params = {
            'g_sdims': 3, 'g_compat': 3, 'b_sdims': 80,
            'b_rgb_sdims': 13, 'b_compat': 10, 'iterations': 10
        }

        print("\n🚀 Iniciando evaluación comparativa (Normal, CRF, TTA) en el conjunto de TEST...")

        num_images = len(glob(os.path.join(EVAL_IMAGES_PATH, '*.jpg')))
        for i, (image_original, image_processed, mask_true_one_hot) in enumerate(eval_dataset):
            print(f"    Procesando imagen {i+1}/{num_images}...", end='\r')
            image_uint8 = tf.cast(image_original[0], tf.uint8).numpy()

            # --- Predicción base ---
            base_probabilities = model.predict(image_processed, verbose=0)[0]

            # --- 1. Predicción Normal (sin filtros) ---
            mask_pred_normal = np.argmax(base_probabilities, axis=-1)
            iou_normal = iou_per_class(mask_true_one_hot, mask_pred_normal[np.newaxis, ...])
            per_class_iou_normal_list.append(iou_normal.numpy())

            # --- 2. Predicción con Post-procesamiento CRF ---
            probs_for_crf = base_probabilities.transpose(2, 0, 1)
            mask_pred_crf = refine_segmentation_with_crf(image_uint8, probs_for_crf, crf_params)
            iou_crf = iou_per_class(mask_true_one_hot, mask_pred_crf[np.newaxis, ...])
            per_class_iou_crf_list.append(iou_crf.numpy())

            # --- 3. Predicción con Test-Time Augmentation (TTA) ---
            probs_pred_tta = perform_segmentation_tta(model, image_processed)
            mask_pred_tta = np.argmax(probs_pred_tta, axis=-1)
            iou_tta = iou_per_class(mask_true_one_hot, mask_pred_tta[np.newaxis, ...])
            per_class_iou_tta_list.append(iou_tta.numpy())

        # Calcular promedios finales y mostrar la tabla de resultados
        avg_iou_normal = np.mean(per_class_iou_normal_list, axis=0)
        avg_iou_crf = np.mean(per_class_iou_crf_list, axis=0)
        avg_iou_tta = np.mean(per_class_iou_tta_list, axis=0)

        print("\n\n" + "---" * 25)
        print(f"📊 Resultados de Evaluación en TEST para: {MODEL_SAVE_PATH}")
        print("---" * 25)
        print(f"{'Clase':<15} | {'IoU Normal':<12} | {'IoU con CRF':<12} | {'IoU con TTA':<12}")
        print("-" * 65)
        for i, class_name in enumerate(CLASS_NAMES):
            print(f"{class_name:<15} | {avg_iou_normal[i]:<12.4f} | {avg_iou_crf[i]:<12.4f} | {avg_iou_tta[i]:<12.4f}")

        print("-" * 65)
        mIoU_normal = np.mean(avg_iou_normal)
        mIoU_crf = np.mean(avg_iou_crf)
        mIoU_tta = np.mean(avg_iou_tta)
        print(f"{'mIoU Promedio':<15} | {mIoU_normal:<12.4f} | {mIoU_crf:<12.4f} | {mIoU_tta:<12.4f}")
        print("---" * 25)

    except Exception as e:
        print(f"\n❌ ERROR: Ocurrió un problema durante la ejecución: {e}")
        traceback.print_exc()

  Cloning https://github.com/lucasb-eyer/pydensecrf.git to /tmp/pip-req-build-hiqkpyg2
  Running command git clone --filter=blob:none --quiet https://github.com/lucasb-eyer/pydensecrf.git /tmp/pip-req-build-hiqkpyg2
  Resolved https://github.com/lucasb-eyer/pydensecrf.git to commit 2723c7fa4f2ead16ae1ce3d8afe977724bb8f87f
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


/usr/local/lib/python3.11/dist-packages/keras/src/layers/layer.py:421: UserWarning: `build()` was called on layer 'deformable_attention', however the layer does not have a `build()` method implemented and it looks like it has unbuilt state. This will cause the layer to be marked as built, despite not being actually built, which may cause failures down the line. Make sure to implement a proper `build()` method.
  warnings.warn(


✅ Modelo 'efficient_weed_model_S_best.keras' cargado exitosamente.
📖 Cargando datos del conjunto de TEST desde: ./Balanced/test/images

🚀 Iniciando evaluación comparativa (Normal, CRF, TTA) en el conjunto de TEST...


---------------------------------------------------------------------------
📊 Resultados de Evaluación en TEST para: efficient_weed_model_S_best.keras
---------------------------------------------------------------------------
Clase           | IoU Normal   | IoU con CRF  | IoU con TTA 
-----------------------------------------------------------------
Background      | 0.9569       | 0.9482       | 0.9582      
Cow-tongue      | 0.9032       | 0.8545       | 0.9043      
Dandelion       | 0.9076       | 0.8378       | 0.9102      
Kikuyo          | 0.9195       | 0.9053       | 0.9107      
Other           | 0.7285       | 0.6854       | 0.7381      
Potato          | 0.9018       | 0.8744       | 0.9042      
--------------------------------------------------------------